# NLP

# 3. Transformers: hype

<img src="attachements/third/nlp_before.png" width="1000"/>

## План занятий

### 1. ЧБДТ: Что Было До Трансформеров
   1. При чём тут нейро-лингвистическое программирование?
   2. Почему вектора?
   3. Счётные методы: CountVectorizer, TF-IDF, добавляем контекст 
   4. Дистрибутивная семантика: Word2vec, FastText
   5. Рекуррентные нейронные сети: LSTM, GRU
### 2. Трансформеры: база
   1. Мотивация
   2. Attention из all you need?: виды attention, интуиция, реализация
   3. Архитектура Transformer: эмбеддинги, энкодер, декодер
### 3. Трансформеры: на волне хайпа
   1. BERT: pre-training, fine-tuning, семейка
   2. GPT: language modelling, n-shot, prompt-engineering, decoding stratetiges
   3. T5: multitask models, seq2seq

## Transformer: архитектура

Исходно Transformer - Encoder-Decoder архитектура для seq2seq задач (машинного перевода).

Задачи в NLP бывают разные, для многих задач архитектура Encoder-Decoder избыточна.

Рассмотрим три основных семейства моделей:
- Encoder (BERT-family) - NLU (natural language understanding) задачи
- Decoder (GPT-family) - NLG (natural language generation) задачи 
- Encoder-Decoder (T5 & co.) - Text2Text задачи

<img src="attachements/third/transformer-arch.png" width="1000"/>

Две стадии: pretraining + finetuning

## BERT: Bidirectional Encoder Representations from Transformers

BERT - encoder-only transformer. Основная задача: генерировать контекстуализированные эмбеддинги для слов и предложения.

Проблемы прошлых моделей:
- Омонимия - значения слов зависят от контекста
- Out-of-vocabulary слова - новые/редкие слова, опечатки
- "Статичность" эмбеддингов

<img src="attachements/third/bert_embeddings.png" width="700"/>

## Pre-training

Идея из word2vec: получить знания о структуре и смысле языка, используя огромные массивы неразмеченных данных.

### Masked language modeling

Похоже на CBOW подход из word2vec.

<img src="attachements/third/cbow.png" width="800"/>

Основные отличия - учитываем связь между словами и весь контекст предложения.

Задача: предсказать слово, которое было скрыто маской на основе всего контекста предложения.

Пайплайн:
- Выбираем 15% токенов
  - в 80% случаев заменяем на токен [MASK]
  - в 10% случаев заменяем на рандомный
  - в 10% случаев оставляем без изменений
- Предсказываем, какой токен был заменен
  - Классификация по всему словарю

<img src="attachements/third/mlm.png" width="1000"/>

### Next sentence prediction

Кроме эмбеддингов слов, мы также получить хороший эмбеддинг предложения. Можно просто усреднить (как мы делали для word2vec), но всегда ли усреднять хорошо?

Задача - предсказать, идут ли предложения друг за другом в тексте?
- Предсказание делается от специального токена [CLS]
- Предложения разделены специальным токеном [SEP]
- 50% предложения следуют друг за другом
- 50% предложения независимы (выбираем случайное из выборки)

<img src="attachements/third/nsp.png" width="1000"/>

## Fine-Tuning

Доучиваем модельку под различные задачи на небольших данных.

### Задачи

#### Classification

Одно предложение - определяем тональность, токсичность или тематику текста.

Пара предложений - определяем связь между предложениями. Например, подходит ли кандидат на вакансию.

##### NLI - Natural Language Inference

Понимаем логическую связь предложений: продолжает, отрицает или не связано.

<img src="attachements/third/nli_task.png" width="800"/>

#### Sequence labelling

Необходимо классифицировать каждый токен в предложении.

Самая частая задача - Named Entity Recognition. Иногда использую как нарицательное.

<img src="attachements/third/ner_task.png" width="800"/>

#### Text segmentation

Задача - выделить границы интересующего нас сегмента текста.

<img src="attachements/third/text_segmentation.png" width="700"/>

#### Как учим BERT на эти задачи?

<img src="attachements/third/bert-tasks.png" width="900"/>

### Токенизация

Три типа токенов:
- Позиционные эмбеддинги - для учета порядка слов
- Эмбеддинги сегментов - для разделения предложений
- Эмбеддинги токенов - привычный слой эмбеддингов

<img src="attachements/third/bert_tokenization.png" width="1000"/>

А в остальном - буквально тот же самый энкодер из Transformer'a.

### Семейка BERT

<table>
<thead>
  <tr>
    <th>Модель</th>
    <th>Саммари</th>
    <th>Описание</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>BERT</td>
    <td>База</td>
    <td>Задачи:<br>- MLM (static masking)<br>- NSP</td>
  </tr>
  <tr>
    <td>RoBERTa</td>
    <td>Улучшенный процесс обучения</td>
    <td>Особенности:<br>- Ооочень много данных (160Gb vs 16Gb)<br>- Большой размер батча (8000 vs 256)<br><br>Задачи:<br>- MLM (dynamic masking)<br>- No NSP</td>
  </tr>
  <tr>
    <td>Electra</td>
    <td>Улучшенный процесс обучения</td>
    <td>Особенности:<br>- Тренинг на всех данных, а не только маскированных<br><br>Задачи:<br>- Replaced token detection (instead of MLM)<br>- No NSP</td>
  </tr>
  <tr>
    <td>Albert</td>
    <td>Облегченный BERT</td>
    <td>Особенности:<br>- Шэринг параметров между слоями (FF / FF+MHA / MHA)<br>- Факторизация эмбеддингов<br><br>Задачи:<br>- MLM<br>- SOP (sentence order prediction)</td>
  </tr>
  <tr>
    <td>DistillBERT &amp; TinyBERT</td>
    <td>Маленькие BERT'ы, полученные за счет дистилляции</td>
    <td>Особенности:<br>- DistillBERT учится предсказывать предсказания большого BERT<br>- TinyBERT учится предсказывать в т.ч. промежуточные слои <img src="attachements/third/distill_vs_tiny.png" width="500"/> </td>
  </tr>
</tbody>
</table>

### Кодим!

#### База

In [1]:
import random
import numpy as np
import torch

def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
seed_everything(0)

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

#### Читаем и готовим датасет - это мы уже видели

In [4]:
with open("data/dataset.txt")as file:
    data = file.read().split("\n")[:-1]
    data = list(map(lambda x: x.split(" ", maxsplit=1), data))
    data = pd.DataFrame(data, columns=["target", "text"])

In [5]:
data.head()

,target,text
0,__label__INSULT,скотина! что сказать
1,__label__NORMAL,я сегодня проезжала по рабочей и между домами ...
2,__label__NORMAL,очередной лохотрон. зачем придумывать очередно...
3,__label__NORMAL,"ретро дежавю ... сложно понять чужое сердце , ..."
4,__label__NORMAL,а когда мы статус агрогородка получили?


In [6]:
data["target"].value_counts()

target
__label__NORMAL                                       203685
__label__INSULT                                        28567
__label__INSULT,__label__THREAT                         6317
__label__THREAT                                         5460
__label__OBSCENITY                                      2245
__label__INSULT,__label__OBSCENITY                      1766
__label__INSULT,__label__OBSCENITY,__label__THREAT       176
__label__OBSCENITY,__label__THREAT                        74
Name: count, dtype: int64

In [7]:
data["target"] = data["target"] \
    .apply(lambda x: "__label__OBSCENITY" if "OBSCENITY" in x else x) \
    .apply(lambda x: "__label__THREAT" if "THREAT" in x else x)

In [8]:
data["target"].value_counts()

target
__label__NORMAL       203685
__label__INSULT        28567
__label__THREAT        11777
__label__OBSCENITY      4261
Name: count, dtype: int64

In [9]:
data = data.sample(frac=1).groupby("target").head(6666).reset_index(drop=True)

In [10]:
data["target"].value_counts()

target
__label__INSULT       6666
__label__NORMAL       6666
__label__THREAT       6666
__label__OBSCENITY    4261
Name: count, dtype: int64

In [11]:
data["target"] = data["target"].factorize()[0]

In [12]:
data.head()

,target,text
0,0,"это твоя мама толстая все сжирает,а у путьки н..."
1,1,правда на 100% . прежде чем что то требовать н...
2,0,"все правильно не уступай трамваю, и поездам то..."
3,2,"таких людей убивать нужно, бедный ребёнок, сер..."
4,1,"опять театр опять развлекуха!! опять клоуны, а..."


In [13]:
data["target"].value_counts()

target
0    6666
1    6666
2    6666
3    4261
Name: count, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_data, test_data = train_test_split(data, test_size=0.15, stratify=data["target"])

In [16]:
X_train, X_test = train_data["text"].values, test_data["text"].values 
y_train, y_test = train_data["target"].values, test_data["target"].values

#### Строим модель!

Huggingface - главная библиотечка для взаимодействия с трансформерами.

Тут есть:
- Хаб предобученных моделей
- Вспомогательные утилиты для обучения и оценки моделей
- И чего только нет...

<img src="attachements/third/huggingface_models.png" width="1000"/>

In [17]:
from transformers import BertTokenizerFast, BertForSequenceClassification

In [18]:
BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 1e-4
NUM_EPOCHS = 5

Берём BERT для классификации последовательностей.

Всё, нужное нам, уже инициализированно. Не забываем файн-тюне.

In [19]:
model = BertForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=data["target"].nunique())
tokenizer = BertTokenizerFast.from_pretrained("cointegrated/rubert-tiny2")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model.to(DEVICE)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

#### Токенизатор и вызов модели

##### Токенизатор

In [21]:
tokenizer("я просто в тебя втюрилась")

{'input_ids': [2, 343, 9922, 314, 29852, 314, 24174, 52655, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
tokenizer.decode([2, 343, 9922, 314, 29852, 314, 24174, 52655, 3])

'[CLS] я просто в тебя втюрилась [SEP]'

In [23]:
tokenizer("я просто в тебя втюрилась", "потому что дора дура")

{'input_ids': [2, 343, 9922, 314, 29852, 314, 24174, 52655, 3, 14665, 1046, 745, 1232, 47802, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenizer.decode([2, 343, 9922, 314, 29852, 314, 24174, 52655, 3, 14665, 1046, 745, 1232, 47802, 3],)

'[CLS] я просто в тебя втюрилась [SEP] потому что дора дура [SEP]'

In [25]:
tokenizer(["я просто в тебя втюрилась", "потому что дора дура"], padding=True, return_tensors="pt")

{'input_ids': tensor([[    2,   343,  9922,   314, 29852,   314, 24174, 52655,     3],
        [    2, 14665,  1046,   745,  1232, 47802,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])}

#### Вызов модели

In [26]:
model(**tokenizer("я просто в тебя втюрилась", return_tensors="pt"))

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0040, -0.0287, -0.0198,  0.0193]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [27]:
model(**tokenizer("я просто в тебя втюрилась", return_tensors="pt"), output_hidden_states=True)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0040, -0.0287, -0.0198,  0.0193]], grad_fn=<AddmmBackward0>), hidden_states=(tensor([[[-0.0142,  0.0298, -0.0084,  ...,  0.0131,  0.0028,  0.0183],
         [-0.9121,  0.3245, -0.3886,  ...,  0.6066,  0.6993, -0.5504],
         [ 1.3372,  0.3197, -0.4329,  ..., -0.1644, -0.7971,  0.4067],
         ...,
         [ 0.9813, -1.7365, -0.0614,  ...,  1.5010, -0.3529,  0.9264],
         [-1.0524, -0.9197,  2.0454,  ..., -0.1242, -0.9972,  1.4023],
         [ 0.1239, -0.2321, -0.0834,  ..., -0.5896, -1.6422, -0.1138]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[-0.0365,  0.1742,  0.2868,  ..., -0.0316,  0.0761, -0.5213],
         [-0.5092, -0.2252,  0.0681,  ..., -0.0459,  0.5087, -0.8478],
         [ 1.8197, -0.0455, -0.6360,  ...,  0.1347, -0.4981, -0.3415],
         ...,
         [ 1.4533, -1.4143, -0.2475,  ...,  1.0662,  0.3260,  0.2414],
         [-0.8736, -1.2774,  1.3637,  ..., -0.3602, -0.5690,  0.6849],
         [-0.

In [28]:
model(**tokenizer("я просто в тебя втюрилась", return_tensors="pt"), labels=torch.tensor(0))

SequenceClassifierOutput(loss=tensor(1.3762, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0040, -0.0287, -0.0198,  0.0193]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

#### Датасет и даталоадер

In [29]:
from torch.utils.data import Dataset, DataLoader
from operator import itemgetter

In [30]:
class BertDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [31]:
def collate(batch):
    texts = list(map(itemgetter(0), batch))
    labels = list(map(itemgetter(1), batch))

    tokens = tokenizer(texts, return_tensors="pt", max_length=64, padding="max_length", truncation=True)
    labels = torch.tensor(labels)

    return tokens, labels

In [32]:
train_dataset = BertDataset(X_train, y_train)
test_dataset = BertDataset(X_test, y_test)

In [33]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate)

#### Тренинг

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred, average='macro'):
    precision = precision_score(y_true, y_pred, average=average)
    recall = recall_score(y_true, y_pred, average=average)
    f1 = f1_score(y_true, y_pred, average=average)

    print(f'{average.capitalize()} Precision = {precision:.4f}, Recall = {recall:.4f}, F1 = {f1:.4f}')

    return precision, recall, f1

In [36]:
def train_epoch():
    model.train()

    losses = []
    for tokens, labels in tqdm(train_dataloader, total=len(train_dataloader)):
        tokens = tokens.to(DEVICE)
        labels = labels.to(DEVICE)

        loss = model(**tokens, labels=labels).loss

        loss.backward()
        losses.append(loss.item())
        optimizer.step()

        optimizer.zero_grad()

    epoch_loss = sum(losses) / len(losses)

    return epoch_loss

In [37]:
@torch.no_grad()
def test_epoch():
    model.eval()

    losses = []
    predictions = []
    for tokens, labels in tqdm(test_dataloader, total=len(test_dataloader)):
        tokens = tokens.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(**tokens, labels=labels)

        pred_labels = outputs.logits.argmax(dim=1).tolist()

        predictions.extend(pred_labels)
        losses.append(outputs.loss.item())

    epoch_loss = sum(losses) / len(losses)
    
    return epoch_loss, predictions

In [38]:
def train_model():
    for epoch in range(NUM_EPOCHS):
        train_loss = train_epoch()
        test_loss, pred_test = test_epoch()

        print(f"TRAIN Epoch {epoch+1}, Loss: {train_loss:.4f}")
        print(f"TEST Epoch {epoch+1}, Loss: {test_loss:.4f}")

        compute_metrics(y_test, pred_test)
        print()
    
    return pred_test 

In [58]:
pred_test = train_model()

100%|██████████| 29/29 [00:05<00:00,  5.59it/s]


TRAIN Epoch 1, Loss: 0.6274
TEST Epoch 1, Loss: 0.3749
Macro Precision = 0.8652, Recall = 0.8634, F1 = 0.8639



100%|██████████| 29/29 [00:05<00:00,  5.66it/s]


TRAIN Epoch 2, Loss: 0.2914
TEST Epoch 2, Loss: 0.3203
Macro Precision = 0.8836, Recall = 0.8850, F1 = 0.8843



  2%|▏         | 4/162 [00:02<01:55,  1.37it/s]


KeyboardInterrupt: 

## GPT: Generative pre-trained transformer

### Language modeling

Language modeling - задача предсказания следующего токена (слова / подслова / буквы) в предложении.

По сути - генерация текста.

Примеры:
- Написание контента - сказок, рассказов, рекламных текстов
- Автодополнение (кода, поискового запроса, текста)

<img src="attachements/third/gpt-2-autoregression.gif" width="1000"/>

GPT - decoder-only transformer, задача которого - авторегрессионно генерировать следущее слово.

### Хитрости и особенности

- Очень много текстов
- Очень (ОЧЕНЬ!) большие модели

<img src="attachements/third/gpt_size.png" width="800"/>

#### Эмерджентность

Большие модели с большими данными обретают новые свойства: учатся решать задачи, которым специально не учились: перевод, ответы на вопросы, NER, сегментация текста, ...

И их больше не нужно файнтюнить!

#### Как заставить модель понять, что от неё нужно?

##### Zero-shot - просто попроси

<img src="attachements/third/zero_shot.png" width="700"/>

##### One-shot - покажи пример

<img src="attachements/third/one_shot.png" width="700"/>

##### Few-shot - покажи несколько примеров

<img src="attachements/third/few_shot.png" width="700"/>

#### Качество возрастает при увеличении модели и количества примеров

<img src="attachements/third/gpt_few_shot.png" width="1000"/>

#### Качество на отдельных задачах

Модель показывает себя очень хорошо либо даже опережает специализированные SOTA решения:

- LAMBADA - нужно предсказать последнее слово в предложении. (нужно улавливать глобальный контекст)
- StoryCloze - нужно выбрать верное окончание истории из четырех вариантов.
- HellaSwag - NLI датасет; нужно выбрать логически подходящее завершение текста.

<img src="attachements/third/gpt_pqrformance.png" width="700"/>

### Prompt-Engineering

Как ещё можно добиться, чтобы модель делала то, что вы от неё хотите?

<img src="attachements/third/prompt_engineering.png" width="600"/>

Подбор "затравки" - очень мощный инструмент. Моделями можно манипулировать!
- Я дам тебе 200 долларов чаевыми, если ты мне поможешь
- Let's think step-by-step
- ...

<img src="attachements/third/prompt_tuning.jpg" width="800"/>

Сейчас - это уже целый мир, всё ограничивается только фантазией.

### Кодим!

#### Читаем модель

In [39]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import json
from pprint import pprint

In [40]:
model = GPT2LMHeadModel.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

In [41]:
VERBOSE = 100
BATCH_SIZE = 20
LR = 1e-5
NUM_EPOCHS = 5

In [42]:
model.to(DEVICE)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [43]:
def tokenize(texts):
    return tokenizer(texts, return_tensors="pt", padding=True)

In [44]:
def generate(prompt, do_sample=False, **kwargs):
    tokens = tokenize(prompt)
    out_tokens = model.generate(**tokens, do_sample=do_sample, **kwargs)

    return tokenizer.batch_decode(out_tokens, ignore_special_tokens=True)

In [99]:
pprint(generate("Шел медведь по лесу", max_length=128))

['Шел медведь по лесу.\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n']


#### Читаем данные

In [46]:
with open("data/anecdotes.json") as file:
    anecdotes = json.load(file)

In [47]:
len(anecdotes)

39636

In [48]:
anecdotes[456]

'Включил я как-то Вечерний ургант выпуск 1290 от 17.04.2020 (на 15:00), а там армяне в нарды играют'

#### Токенизируем немножечко

In [49]:
anecdote_tokens = tokenizer(anecdotes, max_length=2048)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [50]:
lengths = pd.Series(list(map(len, anecdote_tokens["input_ids"])))

In [51]:
lengths.describe(percentiles=[0.5, 0.95, 0.99])

count    39636.000000
mean       118.387552
std        178.214457
min          5.000000
50%         63.000000
95%        369.000000
99%        879.650000
max       2048.000000
dtype: float64

In [52]:
lengths

0         41
1        829
2        136
3        130
4        103
        ... 
39631    459
39632     14
39633     48
39634     28
39635     20
Length: 39636, dtype: int64

In [53]:
len(anecdotes)

39636

In [54]:
anecdotes = [anecdotes[i] for i in lengths[lengths < 350].index]

Отсортируем, чтобы делать меньше паддингов и лишних вычислений.

Мы так уже делали, когда учили LSTM.

In [55]:
anecdotes = sorted(anecdotes, key=lambda x: len(x))

In [56]:
anecdotes[1:3]

['-Ало, это Алена?\n-Да\n-ст', 'Узбек-людоед съел пловца']

#### Датасет и даталоадер

In [57]:
class AnecdoteDataset(Dataset):
    def __init__(self, anecdotes) -> None:
        self.anecdotes = anecdotes
    
    def __len__(self):
        return len(self.anecdotes)
    
    def __getitem__(self, index):
        return self.anecdotes[index]

In [58]:
def anecdote_collate(batch):
    return tokenize(batch)

In [59]:
anecdote_dataset = AnecdoteDataset(anecdotes)
anecdote_dataloader = DataLoader(anecdote_dataset, batch_size=20, shuffle=False, collate_fn=anecdote_collate)

#### А что там в этом батче?

In [60]:
next(iter(anecdote_dataloader))

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,  1140, 11512,   448, 37960,
           367, 15393,     5],
        [   17,   618,   435,    16,   481, 24818,    35,   203,    17,  4529,
           203,    17,   285],
        [    0,     0,     0,  1140,   288, 10810,    17,   623,   660,   383,
         37284, 38607,   613],
        [    0,     0,     0,    17,  1140,  1304,  8294, 23647,    35,   203,
            17,  4529,    18],
        [    0,     0,     0,     0,     0,   822, 20681,  4527,   802,   282,
          3808,   476,    18],
        [    0,     0,     0,     0,     0,     0,  6115,  1496,  7999, 40571,
           309, 32679,     8],
        [    0,     0,    17,   548,   374,    16,  9160,   356,  6036,    35,
           203,    17,   347],
        [    0,     0,     0,     0,     0,     0,     0,     0,  2997,   798,
          6249, 26133,  4038],
        [    0,     0,     0,  7148,  1594,  4961,    16,   583,   694,   279,
           578,   350,    1

Заметим:
1. Паддинг слева
2. Паддится не так много (но можно и меньше: как?)

#### Обучение

In [61]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [62]:
def train_epoch(epoch):
    model.train()
    losses = []

    for i, input_tokens in enumerate(tqdm(anecdote_dataloader, total=len(anecdote_dataloader))):
        input_tokens = input_tokens.to(DEVICE)

        labels = input_tokens["input_ids"].clone()
        labels[labels == 0] = -100  # to ignore them when computing loss
        
        loss = model(
            **input_tokens,
            labels=labels,
            return_dict=True
        ).loss

        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        if (i + 1) % VERBOSE == 0:
            print(f"Step {i + 1}: loss - {sum(losses[-VERBOSE:-1]) / VERBOSE}")
            model.eval()
            print(generate("Шел медведь по лесу", max_length=128))
            model.train()

    epoch_loss = sum(losses) / len(losses)

    print(f"EPOCH {epoch} - epoch loss: {(epoch_loss):3f}")
    torch.cuda.empty_cache()

    return epoch_loss

In [63]:
train_epoch(0)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 1/1871 [00:01<34:45,  1.12s/it]

  5%|▌         | 99/1871 [01:51<32:10,  1.09s/it]  

Step 100: loss - 5.522265133857727


  5%|▌         | 100/1871 [01:56<1:04:29,  2.18s/it]

Шел медведь по лесу.
- А ты? - спросил он у меня, когда я ему ответил: "Я не знаю".
- Я не знаю! 
- Ты знаешь, что это за зверь?
- Нет.
- Это волк.
- Волк?!
- Да.
- И как его зовут?
- Медведь.
- Как волка?
- Волком.
- Ну да.
- Он же в пальто.
- В пальто.
- Неправда.
- Почему нет?
- Потому что он на мне.
- Что значит "в пальто"?
-


...

Step 200: loss - 4.893615212440491


 11%|█         | 200/1871 [04:10<1:16:27,  2.75s/it]

Шел медведь по лесу.
- Да, да! - закричал в ответ волк.
- А я не могу его убить медведя?
- Нет.
- Ну и что ты ему сказал?
- Я тебе говорю:
- Ты мне говорил, а он говорит:
- Не надо меня убивать медведя?
- Так это же медведь может только на тебя.
- Мммм...
- И как вы думаете, кто там был?
- Медведя?
- Он сам себя убил медведя?
- Нет.
- Тогда давай, ну и так далее.
- Это было бы круто.



 11%|█         | 205/1871 [04:17<34:54,  1.26s/it]  
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [64]:
model.eval();

##### Decoding strategies

Трейдофф между связностью и разнообразием.

##### Greedy decoding

На каждом шаге генерации выбираем самый вероятный токен.

Получаем:
- очень связный текст
- невозможность получения разнообразных последовательностей

<img src="attachements/third/greedy_decoding.webp" width="600"/>

In [71]:
pprint(
    generate("Шел медведь по лесу", max_length=64)
)

('Шел медведь по лесу.\n'
 '- Да, да! - закричал в ответ.\n'
 '- А я не могу.\n'
 '- Не можешь?\n'
 '- Нет.\n'
 '- Ну и что ты делаешь?\n'
 '- Я тебе говорю: «Я тебя люблю».\n'
 '- Ты меня любишь?\n'
 '- Люблю.\n'
 '-')


##### Sampling

##### Top-k sampling

Случайно выбираем из первых К предложенных вариантов согласно предсказанным вероятностям токенов.

<img src="attachements/third/topk_sampling.webp" width="600"/>

In [70]:
pprint(
    generate("Шел медведь по лесу", max_length=64, do_sample=True, top_k=3)
)

('Шел медведь по лесу. \n'
 '- Я не знаю, но я люблю его... \n'
 '- Да! - и тут же: - У него есть жена? \n'
 '\n'
 '- Нет. \n'
 '- А он любит его. \n'
 '\n'
 'И вот они вместе в лес идут. \n'
 '\n'
 'В лесу. \n'
 '-')


##### Softmax temperature

Способ сделать распределение более равномерным.

Таким образом, выбор в Top-k семплинге будет более разнообразный.

<img src="attachements/third/softmax_temperature.jpg" width="800"/>

Чам больше температура, тем более равномерное распределение получается.

In [93]:
pprint(
    generate("Шел медведь по лесу",  max_length=64, do_sample=True, top_k=3, temperature=2.)
)

('Шел медведь по лесу, а он ему в лицо:\n'
 '\n'
 '—\xa0Бля, ты не видишь, как я тебя люблю.\n'
 '—\xa0Давай я.\n'
 '—\xa0Я тебя люблю.\n'
 '—\n'
 '\n'
 '—\xa0Да, да.\n'
 '-\n'
 '—\n'
 '—\n'
 '—\n'
 '-\n'
 '-')


##### Проблемка

Хочется сделать К динамическим: иногда вариантов очень много, а иногда - мало.

<img src="attachements/third/topk_problem.png" width="600"/>

##### Top-p sampling

К зависит от суммы вероятностей К предсказанных токенов.

<img src="attachements/third/top_p_sampling.webp" width="600"/>

In [94]:
pprint(
    generate("Шел медведь по лесу", max_length=64, do_sample=True, top_p=0.7, temperature=2.)
)

('Шел медведь по лесу, а за ним в лесная глу, там где она впадает озеро. Я в '
 'реке. У меня нет желания в лес.\n'
 '- Но я не туда, как-нибудь вы там в другой раз на этот же остров.\n'
 '-  -  -   -- - ')


##### Beam search

Прошлые методы выбирают следующий токен без учета контекста.

Beam search - строит несколько вариантов генераций и выбирает самую вероятную *последовательность* токенов.

<img src="attachements/third/beam_search.webp" width="800"/>

Это требует больше вычислений, но результаты получаются намного лучше.

In [104]:
pprint(
    generate(
        "Шел медведь по лесу", 
        max_length=64, 
        num_beams=5,
        no_repeat_ngram_size=2,
        num_return_sequences=3,
    )
)

['Шел медведь по лесу.\n'
 '\n'
 '—\xa0Да,\xa0— сказал он, и я понял, что он говорит по-русски, а я не знаю '
 'русского языка, но я знаю, как его зовут. Я не понимаю, я его не люблю. А '
 'ты, значит, не любишь русских?\n'
 '-\xa0',
 'Шел медведь по лесу.\n'
 '\n'
 '—\xa0Да,\xa0— сказал он, и я понял, что он говорит по-русски, а я не знаю '
 'русского языка, но я знаю, как его зовут. Я не понимаю, я его не люблю. А '
 'ты, значит, не любишь?\n'
 '- Нет,',
 'Шел медведь по лесу.\n'
 '\n'
 '—\xa0Да,\xa0— сказал он, и я понял, что он говорит по-русски, а я не знаю '
 'русского языка, но я знаю, как его зовут. Я не понимаю, я его не люблю. А '
 'ты, значит, не любишь?\n'
 '- Нет.']


In [114]:
pprint(
    generate(
        "Шел медведь по лесу", 
        max_length=64, 
        num_beams=5,
        no_repeat_ngram_size=2,
        num_return_sequences=3,
        do_sample=True,
        top_p=0.75,
        temperature=2.
    )
)

['Шел медведь по лесу.\n'
 '\n'
 '- Да, да. \n'
 'Мало кто знает, что у него есть два брата?\n'
 '(с)\n'
 '"Мальчик в лесу".\n'
 'О, нет, это не мой брат, а его брат-брат-бабочка-с-поломанной',
 'Шел медведь по лесу.\n'
 '\n'
 '- Да, да. \n'
 'Мало кто знает, что у него есть два брата?\n'
 '(с)\n'
 '"Мальчик в лесу".\n'
 'О, нет, это не мой брат, а его брат-брат-бабочка-с-покойник',
 'Шел медведь по лесу.\n'
 '\n'
 '- Да, да. \n'
 'Мало кто знает, что у него есть два брата?\n'
 '(с)\n'
 '"Мальчик в лесу".\n'
 'О, нет, это не мой брат, а его брат-брат-бабочка-с-покойнич']


## T5: Transfer learning in all its glory

А что если объединить BERT и GPT и решать оооочень много различных задач?

Получится T5 - Encoder-Decoder transformer, предназначенный для решения text2text задач.

Можно представлять различные задачи в виде Text2text:

<img src="attachements/third/t5_multitask.png" width="1000"/>

### Предобучение

Почти как BERT, только сложнее: маскируем последовательности слов.

<img src="attachements/third/t5_pretraining.png" width="1000"/>

### Обучение

Задачи:
- Перевод - переводим с одного языка на другой.
- Суммаризация - делаем краткий пересказ текста.
- Question answering - отвечаем на вопрос по тексту.
- Классификация текстов - относим текст к какой-то группе.
- Генерация текстов - сочиняем (продолжаем) текст.
- Регрессия по текстам - предсказываем число по тексту (анализ тональности, возраст, ...). 
- NLI (Natural language inference) - понимаем логическую связь предложений: продолжает, отрицает или не связано.
- Сегментация текстов - находим отрезки текста, где что-то важное нам происходит (аспектный анализ тональности, ответ на вопрос, ...)

*(Т5 училась не на все эти задачи, некоторые просто как пример.)*

<img src="attachements/third/t5_tasks.png" width="1000"/>

### Encoder-Decoder vs Decoder-only

- Но ведь GPT умеет делать то же самое!
- Да, что-то из этого может и BERT делать...

<img src="attachements/third/transformer-arch.png" width="1000"/>

#### Encoder-only

Всякие BERT, RoBERTa and co.

Любые NLU-only задачи:
- Text Classification
- Sequence labelling
- Sentiment analysis
- NLI
- QA
- Search & Recommender systems (спойлер!)

#### Decoder-only

Всякие GPT, XLNet and co.

Любые чисто генеративные задачи:
- Text completion
- Text generation
- Creative writing

И задачи, где не требуется глубокое понимание:
- диалоговые системы

#### Encoder-Decoder

Всякие Transformer, BART, T5 and co.

Задачи на понимание + генерацию:
- Machine translation
- Summarization
- QA

#### Реальность

Берем ChatGPT и погнали...

### Кодим!

In [1]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from datasets import load_dataset

/Users/aleksiej/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = T5TokenizerFast.from_pretrained("cointegrated/rut5-small")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/aleksiej/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
You are using a

У huggingface, кроме моделей, есть и датасеты.

<img src="attachements/third/huggingface_datasets.png" width="1000"/>

In [3]:
datasets = load_dataset('IlyaGusev/gazeta', revision="v2.0")

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
})

In [5]:
datasets["train"][11]

{'text': '«Ставки, которые есть сейчас в экономике, не отражают реальную оценку рисков. Риски в экономике значительно выше, чем это было в 2006, 2007, 2008 годах, а ставки значительно ниже», — заявил глава Сбербанка Герман Греф на форуме «Стратегия-2010, итоги реализации и новые вызовы». В связи с этим, по его мнению, банки будут иметь либо очень низкую, либо отрицательную маржу, что «будет влиять на выздоровление банковского сектора, который находится в очень непростой ситуации». «Низкими ставками мы загоним инвесторов в инвестпроекты. А когда тренд будет «откручиваться» обратно, у нас появится новое банкротство компаний и банков», — добавил Греф. В ближайшие годы может появиться большое количество нереализованных проектов, которые могут оказаться в том числе и на балансах банков, считает глава Сбербанка. Мнение Грефа идет вразрез с мнением российских властей, которые регулярно призывают банки сделать заемные средства доступными для бизнеса и граждан. «Мы должны последовательно занима

In [6]:
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

In [7]:
def preprocess_data(examples):
    model_inputs = tokenizer(examples["text"], max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["title"], 
            max_length=MAX_TARGET_LENGTH, 
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [8]:
datasets = datasets.map(preprocess_data, batched=True)

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6369
    })
})

In [10]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

<img src="attachements/third/seq2seq_arguments.png" width="800"/>

In [15]:
args = Seq2SeqTrainingArguments(
    "t5_small_absum",
    evaluation_strategy="steps",
    eval_steps=20,
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    save_steps=20,
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=1,
    predict_with_generate=True,
    load_best_model_at_end=True,
    report_to=None
)

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [17]:
trainer = Seq2SeqTrainer(
    model_init=lambda: T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small"),
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [18]:
trainer.train()

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.





















                                                   

                                             
  0%|          | 20/7621 [02:19<3:48:17,  1.80s/it]


{'loss': 3.2923, 'learning_rate': 3.989502689935704e-05, 'epoch': 0.0}


KeyboardInterrupt: 

In [19]:
model_dir = "t5_small_absum/"

tokenizer = T5ForConditionalGeneration.from_pretrained(model_dir)
model = T5TokenizerFast.from_pretrained(model_dir)

max_input_length = 512

OSError: t5_small_absum/ does not appear to have a file named config.json. Checkout 'https://huggingface.co/t5_small_absum//main' for available files.

In [26]:
text_to_summarize = datasets["test"][0]["text"]
text_to_summarize

'На этих выходных в Берлине прошли крупные акции протеста против введенных для борьбы с коронавирусом ограничений. Демонстранты скандировали «Путин!» По словам депутата городской палаты представителей Гуннара Линдеманна («Альтернатива для Германии»), люди выкрикивали фамилию российского президента из уважения к нему. В комментарии РИА «Новости» немецкий политик отметил, что среди населения Германии Владимир Путин имеет хорошую репутацию. По его мнению, протестующие ранее пришли к российскому посольству, чтобы «привлечь внимание к условиям в Германии», надеясь, что Россия сможет оказать влияние на канцлера ФРГ Ангелу Меркель. «На мой взгляд, опасности для посольства России не возникло ни разу», — сказал депутат. Несмотря на то что протест оказался массовым, выступления носили «преимущественно мирный характер», уверен Линдеманн. По его словам, исключением стала только ситуацию у немецкого парламента. Там «несколько странных участников демонстрации попытались штурмовать бундестаг в знак п

In [32]:
inputs = tokenizer(text_to_summarize, max_length=MAX_INPUT_LENGTH, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[ 1051,  4000,   777,  6652,  1140,   315, 14109,  5842,   923,  5875,
          9272,  1293,   259, 10137,  8549,   308,  5439,   315,  8385,  1140,
           259,   735,  5145,   833,  2314,   388, 13724, 12615,   637,  8377,
          1008,   260,  5209, 17628, 10536, 19293, 12207,   404,  8134,  7271,
          1325,  5705,   507,  6229,   308,  5469,  5004,  9647,   433,  7595,
          1011,  7155, 11308,  4531, 11666,  7404, 14892,  7598, 18070,  4637,
          8157,  1789,   259,   735, 11115,   279, 14012,  5610, 19521,  7434,
          6585,   259, 13170,   748,   259,  6739,   893,  5805,   308,   729,
          9937,  2013,   778,   259,  9859,   260,   635, 11034,   279,  1936,
         10427,   404, 14597,   436,  7647, 13311,  7668,   552, 10084,   261,
           892,  5138,   279,  6329,  1419, 11115,   279,  6614,   259,  8134,
          4338,  1348,  5759,  2536,  3613,  8149,  9327,   260,  1325,   259,
          1802,  5298,  6863,   261,  

In [45]:
output = model.generate(**inputs.to("mps"), num_beams=4, do_sample=True, min_length=10, max_length=64)

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [40]:
output

tensor([[    0,   635, 11115,   279,  8549,  7799,   425,  7739,  5862,   259,
         14781,   507,  6158, 11092,  1704,  5984,   909,   261,   388,  2021,
          5891,   259,  3266,   805, 10025,  5158,  1592,   315,   401, 12193,
           817,  9647,   324,  7595,  1011, 11115,   279,   261,   401,  6283,
           396,   310,   259,  3205,  5705,   507,   261,   401,  6283,   396,
           310,   259,  1802,  5705,   507,   261,   805, 10025,  5158,  1592,
           315,   259, 10137,  8549]], device='mps:0')

In [39]:
tokenizer.batch_decode(output, skip_special_tokens=True)[0]

'В Германии протестующие попытались штурмовать бундестаг, с которыми они приняли участие в немецкой палате представителей Германии, несмотря на ее словам, несмотря на его словам, приняли участие в акции протест'

## Bye

<img src="attachements/second/thanks_attention.png" width="1200"/>